In [5]:
#softmax classification(multinimial logistic)
import tensorflow as tf
tf.set_random_seed(777)


x_data = [[1, 2, 1, 1],
          [2, 1, 3, 2],
          [3, 1, 3, 4],
          [4, 1, 5, 5],
          [1, 7, 5, 5],
          [1, 2, 5, 6],
          [1, 6, 6, 6],
          [1, 7, 7, 7]]
y_data = [[0, 0, 1],
          [0, 0, 1],
          [0, 0, 1],
          [0, 1, 0],
          [0, 1, 0],
          [0, 1, 0],
          [1, 0, 0],
          [1, 0, 0]]

X = tf.placeholder("float", [None, 4])
Y = tf.placeholder("float", [None, 3])
nb_classes = 3

w = tf.Variable(tf.random_normal([4, nb_classes]), name = 'weight')
b = tf.Variable(tf.random_normal([nb_classes]), name = 'bias')

hypothesis = tf.nn.softmax(tf.matmul(X,w) +b)

cost = tf.reduce_mean(-tf.reduce_sum(Y*tf.log(hypothesis), axis = 1))

optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

In [8]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(2001):
        sess.run(optimizer, feed_dict = {X : x_data, Y : y_data})
        if step % 200 == 0:
            print(step, sess.run(cost, feed_dict = {X : x_data, Y : y_data}))
            
    print('------------')
    
    a = sess.run(hypothesis, feed_dict = {X : [[1,11,7,8]]})
    print(a, sess.run(tf.argmax(a,1)))
    
    print('------------')
    
    b = sess.run(hypothesis, feed_dict = {X : [[1,3,4,3]]})
    print(b, sess.run(tf.argmax(b,1)))
    
    print('------------')
    
    c = sess.run(hypothesis, feed_dict = {X : [[1,1,0,1]]})
    print(c, sess.run(tf.argmax(c,1)))
    
    print('------------')
    
    all = sess.run(hypothesis, feed_dict = {X : [[1,11,7,9], [1,3,4,3], [1,1,0,1]]})
    print(all, sess.run(tf.argmax(all,1)))

0 4.0164614
200 0.5666292
400 0.4542002
600 0.35720807
800 0.2680344
1000 0.2316756
1200 0.2098048
1400 0.19166166
1600 0.17634565
1800 0.1632396
2000 0.15189894
------------
[[5.379780e-02 9.461517e-01 5.050248e-05]] [1]
------------
[[0.9134662  0.07969136 0.00684246]] [0]
------------
[[1.0958090e-08 3.2652201e-04 9.9967349e-01]] [2]
------------
[[2.06206716e-03 9.97929335e-01 8.55135659e-06]
 [9.13466215e-01 7.96913579e-02 6.84245909e-03]
 [1.09580895e-08 3.26522015e-04 9.99673486e-01]] [1 0 2]


In [11]:
#실제 데이터를 이용한 softmax classfication
import tensorflow as tf
import numpy as np
tf.set_random_seed(777)

xy = np.loadtxt('C:/Users/admin/Desktop/새홀리기/data-04-zoo.csv', delimiter = ',', dtype = np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, [-1]]
print(x_data.shape, y_data.shape)

(101, 16) (101, 1)


In [18]:
# Predicting animal type based on various features
nb_classes = 7  # 0 ~ 6

X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.int32, [None, 1])  # 0 ~ 6
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot
print("one_hot", Y_one_hot)
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])
print("reshape", Y_one_hot)

W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits,
                                                 labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


one_hot Tensor("one_hot_2:0", shape=(?, 1, 7), dtype=float32)
reshape Tensor("Reshape:0", shape=(?, 7), dtype=float32)
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [19]:
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(2000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 100 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={
                                 X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(
                step, loss, acc))

    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    for p, y in zip(pred, y_data.flatten()):
        print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))


Step:     0	Loss: 5.052	Acc: 3.96%
Step:   100	Loss: 0.749	Acc: 80.20%
Step:   200	Loss: 0.411	Acc: 88.12%
Step:   300	Loss: 0.294	Acc: 89.11%
Step:   400	Loss: 0.231	Acc: 93.07%
Step:   500	Loss: 0.190	Acc: 95.05%
Step:   600	Loss: 0.161	Acc: 98.02%
Step:   700	Loss: 0.139	Acc: 98.02%
Step:   800	Loss: 0.122	Acc: 98.02%
Step:   900	Loss: 0.109	Acc: 99.01%
Step:  1000	Loss: 0.099	Acc: 99.01%
Step:  1100	Loss: 0.090	Acc: 100.00%
Step:  1200	Loss: 0.083	Acc: 100.00%
Step:  1300	Loss: 0.077	Acc: 100.00%
Step:  1400	Loss: 0.071	Acc: 100.00%
Step:  1500	Loss: 0.067	Acc: 100.00%
Step:  1600	Loss: 0.063	Acc: 100.00%
Step:  1700	Loss: 0.059	Acc: 100.00%
Step:  1800	Loss: 0.056	Acc: 100.00%
Step:  1900	Loss: 0.054	Acc: 100.00%
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 Tru